# Using ActionWeaver with Pydantic V2
This guide is designed to demonstrate use cases using Pydantic along with ActionWeaver.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

from typing import List
from uuid import UUID, uuid4

from actionweaver import action
from actionweaver.utils.tokens import TokenUsageTracker
from actionweaver.llms import wrap
from actionweaver.actions.factories.pydantic_model_to_action import  action_from_model

from pydantic import BaseModel, Field, PrivateAttr, validate_call

from openai import OpenAI

from datetime import datetime

Prerequisite: Patch OpenAI client with ActionWeaver

In [ ]:
client = wrap(OpenAI())

Use Case: Structure parsing using Pydantic V2

In [ ]:
class User(BaseModel):
    _uid: UUID = PrivateAttr(default_factory=uuid4)
    _created_at: datetime = PrivateAttr(default_factory=datetime.now)
    name: str = Field(..., description="Name of the user")
    age: int = Field(..., description="Age of the user", gt=0)

class Users(BaseModel):
    users: List[User]

create_users = action_from_model(Users, stop=True)
create_users.json_schema()

In [ ]:
users = create_users.invoke(client, messages=[{"role": "user", "content": "Niel and Tom are 31 years old, James is 25 years old"}], model="gpt-3.5-turbo", stream=False, force=False)

In [ ]:
users

Use Case: You can also use Pydantic models as function parameter.

The `@validate_call` decorator allows the arguments passed to a function to be parsed and validated using the function's annotations before the function is called.

In [ ]:
user_db = []

@action(name="InsertUsers", stop=True)
@validate_call
def insert_users(users: Users) -> str:
    """Inserts user data into the database and returns a success message."""
    # Add the provided users to the user database
    user_db.extend(users.users)
    
    return f"Inserted {len(users.users)} users ."

In [ ]:
messages = [
    {"role": "user", "content": "Add following users: Niel and Tom are 31 years old, James is 25 years old"}
  ]

response = client.create(
    model="gpt-3.5-turbo",
    messages=messages,
    actions = [insert_users],
    stream=False, 
    token_usage_tracker = TokenUsageTracker(500),
)

response, user_db